In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.models as models
import torch.nn.functional as F
from PIL import Image
import os
import numpy as np
import cv2
from tqdm import tqdm


from matplotlib.pyplot import imshow
import math
import json
import re
from scipy.ndimage import gaussian_filter1d, gaussian_filter

In [2]:
# Configuration
# device = torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "best_VGG_model_1.pth"  # Update with your model path

In [3]:
class VGG16BinaryClassifier(nn.Module):
    def __init__(self, pretrained=True):
        super(VGG16BinaryClassifier, self).__init__()
        self.vgg16 = models.vgg16(pretrained=pretrained)
        for param in self.vgg16.features.parameters():
            param.requires_grad = True
        self.vgg16.classifier = nn.Sequential(
            nn.Linear(25088, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        return self.vgg16(x)

In [4]:
class OcclusionSensitivity:
    def __init__(self, model, window_size=32, stride=16):
        self.model = model
        self.window_size = window_size
        self.stride = stride
        
    def generate_heatmap(self, input_tensor, target_class=None):
        """Generate occlusion sensitivity heatmap"""
        with torch.no_grad():
            # Original prediction
            original_output = torch.sigmoid(self.model(input_tensor))
            original_prob = original_output.item()
            original_class = 1 if original_prob > 0.5 else 0
            target_class = target_class or original_class
            
            # Setup dimensions
            b, c, h, w = input_tensor.shape
            heatmap = torch.zeros((h, w), device=device)
            pad = self.window_size // 2
            
            # Pad input for complete coverage
            padded_input = F.pad(input_tensor, (pad, pad, pad, pad), value=0)
            
            # Slide occlusion window
            for y in tqdm(range(0, h, self.stride), desc="Generating occlusion map"):
                for x in range(0, w, self.stride):
                    # Create occluded version
                    occluded = padded_input.clone()
                    y_start = y + pad
                    x_start = x + pad
                    occluded[..., y_start:y_start+self.window_size, 
                            x_start:x_start+self.window_size] = 0
                    
                    # Get modified prediction
                    output = torch.sigmoid(self.model(occluded[..., pad:-pad, pad:-pad]))
                    current_prob = output.item()
                    
                    # Calculate impact score
                    if target_class == 1:
                        score = original_prob - current_prob
                    else:
                        score = current_prob - original_prob
                    
                    # Update heatmap
                    y_end = min(y + self.stride, h)
                    x_end = min(x + self.stride, w)
                    heatmap[y:y_end, x:x_end] += score
            
            # Normalize and return
            heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min() + 1e-8)
            return heatmap.cpu().numpy()

In [5]:
def load_model(model_path, device):
    model = VGG16BinaryClassifier(pretrained=True)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device).eval()
    return model

In [6]:
# Process a directory of B-scans with occlusion sensitivity
def process_bscans_with_occlusion(model, bscan_dir, output_dir, transform, window_size=32, stride=16):
    """
    Process all B-scan images in a directory with occlusion sensitivity and save results
    
    Args:
        model: The trained model
        bscan_dir: Directory containing B-scan images
        output_dir: Directory to save occlusion heatmaps
        transform: Image transformation for model input
        window_size: Size of occlusion window
        stride: Stride for occlusion analysis
    """
    os.makedirs(output_dir, exist_ok=True)
    
    # Create occlusion sensitivity analyzer
    occluder = OcclusionSensitivity(model, window_size=window_size, stride=stride)
    
    # Get list of images
    img_filenames = sorted([
        f for f in os.listdir(bscan_dir)
        if f.endswith((".png", ".jpg", ".jpeg"))
    ])
    
    for img_name in tqdm(img_filenames, desc="Processing B-scans"):
        img_path = os.path.join(bscan_dir, img_name)
        
        # Load and process image
        image = Image.open(img_path).convert('RGB')
        input_tensor = transform(image).unsqueeze(0).to(device)
        
        # Generate heatmap
        heatmap = occluder.generate_heatmap(input_tensor)
        
        # Process visualizations
        image_np = np.array(image)
        heatmap_resized = cv2.resize(heatmap, (image_np.shape[1], image_np.shape[0]))
        
        # Create heatmap overlay
        heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap_resized), cv2.COLORMAP_INFERNO)
        heatmap_colored = cv2.cvtColor(heatmap_colored, cv2.COLOR_BGR2RGB)
        
        # Blend with original image
        blended = cv2.addWeighted(image_np, 0.7, heatmap_colored, 0.3, 0)
        
        # Save results
        output_path = os.path.join(output_dir, img_name)
        cv2.imwrite(output_path, cv2.cvtColor(blended, cv2.COLOR_RGB2BGR))
        
        # Save raw heatmap for further processing
        heatmap_path = os.path.join(output_dir, f"heatmap_{img_name}")
        cv2.imwrite(heatmap_path, np.uint8(255 * heatmap_resized))
    
    print(f"✅ Processed {len(img_filenames)} B-scans with occlusion sensitivity")

In [7]:
def list_folders(bscan_dir):
    """List all folders in the B-scan directory with error handling"""
    bscan_path = Path(bscan_dir)
    if not bscan_path.exists():
        return f"Error: Directory '{bscan_dir}' does not exist"
    if not bscan_path.is_dir():
        return f"Error: '{bscan_dir}' is not a directory"
    return [f.name for f in bscan_path.iterdir() if f.is_dir()]

In [10]:
import os
from pathlib import Path
def main():
    # Configuration
    model_path = "best_VGG_model_1.pth"
    bscan_dir = "BScan"  # Directory containing B-scan images
    occlusions_output_dir = "img_masked_img"  # Directory to save occlusion heatmaps
    enface_path = "enface.jpg"  # Path to the enface image
    folders = list_folders(bscan_dir)

    for folder in folders:
        src_folder = os.path.join(bscan_dir, folder)
        dest_folder = os.path.join(occlusions_output_dir, folder)
        src_folder = os.path.join(src_folder, "predictions", "fovea")
        print(src_folder)
        # Define transforms
        transform = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

        # Load model
        model = load_model(model_path, device)
        try:
            #check if the destination folder exists and is empty
            if os.path.exists(dest_folder):
                if len(os.listdir(dest_folder)) > 0:
                    continue
            # Process B-scans with occlusion sensitivity
            process_bscans_with_occlusion(
                model=model,
                bscan_dir=src_folder,
                output_dir=dest_folder,
                transform=transform,
                window_size=32,
                stride=16
            )
        except Exception as e:
            print(f"Error processing folder '{folder}': {e}")

In [11]:
if __name__ == "__main__":
    main()

BScan/1003_right/predictions/fovea
BScan/1025_right/predictions/fovea
BScan/1007_left/predictions/fovea
BScan/1019_left/predictions/fovea
BScan/1004_right/predictions/fovea
BScan/1010_right/predictions/fovea
BScan/1021_right/predictions/fovea
BScan/1014_right/predictions/fovea
BScan/1019_right/predictions/fovea
BScan/1013_left/predictions/fovea
BScan/healthy_no/predictions/fovea
BScan/fovea_no/predictions/fovea
BScan/1028_left/predictions/fovea
BScan/1011_left/predictions/fovea
BScan/1032_right/predictions/fovea
BScan/1005_right/predictions/fovea
BScan/1024_right/predictions/fovea
BScan/1008_left/predictions/fovea
BScan/1011_right/predictions/fovea
BScan/1026_right/predictions/fovea
BScan/1002_left/predictions/fovea
BScan/1022_left/predictions/fovea
Error processing folder '1022_left': [Errno 2] No such file or directory: 'BScan/1022_left/predictions/fovea'
BScan/1017_left/predictions/fovea


Processing B-scans: 100%|██████████| 22/22 [05:14<00:00, 14.28s/it]


✅ Processed 22 B-scans with occlusion sensitivity
BScan/1029_right/predictions/fovea


Processing B-scans: 100%|██████████| 17/17 [03:45<00:00, 13.28s/it]


✅ Processed 17 B-scans with occlusion sensitivity
BScan/fovea_yes/predictions/fovea


...

✅ Processed 20 B-scans with occlusion sensitivity
BScan/1004_left/predictions/fovea


Processing B-scans: 100%|██████████| 12/12 [02:30<00:00, 12.55s/it]


✅ Processed 12 B-scans with occlusion sensitivity
BScan/1015_right/predictions/fovea


Processing B-scans: 100%|██████████| 17/17 [03:30<00:00, 12.40s/it]

✅ Processed 17 B-scans with occlusion sensitivity
